#**Reliable JSON Output Generation with LLM, Parsing the JSON output for error handling & Pydantic (for JSON Schema Validation)**

##**0. Install Dependencies**

In [1]:
!pip install pydantic openai

##**1. Import Required Modules**

In [11]:
import json
from typing import Dict
from pydantic import BaseModel, ValidationError, RootModel
from openai import OpenAI

##**2. Get API key from Secret**

In [12]:
from google.colab import userdata
# Retrieve the API key from Colab's secrets
api_key = userdata.get('OPENAI_API_KEY')

##**3. Set Up OpenAI Client**

In [13]:
# Set your OpenAI API key
client = OpenAI(api_key=api_key)

##**4. Define Pydantic Models**

In [20]:
class PersonInfo(BaseModel):
    name: str
    age: int
    city: str

# Using RootModel to support JSON like {"employee1": {...}, "employee2": {...}}
class EmployeeDict(RootModel[Dict[str, PersonInfo]]):
    pass


##**5. Compose Prompt and Call GPT**
- JSON Output Generation
- Parsing the JSON output for error handling
- Schema validation

In [23]:
#JSON Output Generation
prompt = """
Generate name, age, and city of 5 employees like employee1, employee2 etc.
Respond ONLY with a JSON object.
"""

try:
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    output = response.choices[0].message.content.strip()
    print("Raw output:\n", output)

    try:
        #Parsing the JSON output for error handling
        data = json.loads(output)
        # Schema validation
        validated_data = EmployeeDict.model_validate(data)
        print("\n✅ Validated Data:\n", validated_data.model_dump())
    except json.JSONDecodeError as e:
        print("❌ JSON parsing failed:", e)
    except ValidationError as ve:
        print("❌ Validation failed:", ve)

except Exception as e:
    print("OpenAI API error:", e)

Raw output:
 {
  "employee1": {
    "name": "John Doe",
    "age": 35,
    "city": "New York"
  },
  "employee2": {
    "name": "Jane Smith",
    "age": 40,
    "city": "Los Angeles"
  },
  "employee3": {
    "name": "Mike Johnson",
    "age": 45,
    "city": "Chicago"
  },
  "employee4": {
    "name": "Emily Davis",
    "age": 30,
    "city": "Houston"
  },
  "employee5": {
    "name": "William Brown",
    "age": 50,
    "city": "Phoenix"
  }
}

✅ Validated Data:
 {'employee1': {'name': 'John Doe', 'age': 35, 'city': 'New York'}, 'employee2': {'name': 'Jane Smith', 'age': 40, 'city': 'Los Angeles'}, 'employee3': {'name': 'Mike Johnson', 'age': 45, 'city': 'Chicago'}, 'employee4': {'name': 'Emily Davis', 'age': 30, 'city': 'Houston'}, 'employee5': {'name': 'William Brown', 'age': 50, 'city': 'Phoenix'}}


##**(Optional) Generate Invalid Data and check Validations**

In [24]:
#JSON Output Generation
prompt = """
Generate name, age, and city of 5 employees.
Respond ONLY with a JSON object.
"""

try:
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    output = response.choices[0].message.content.strip()
    print("Raw output:\n", output)

    try:
        #Parsing the JSON output for error handling
        data = json.loads(output)
        # Schema validation
        validated_data = EmployeeDict.model_validate(data)
        print("\n✅ Validated Data:\n", validated_data.model_dump())
    except json.JSONDecodeError as e:
        print("❌ JSON parsing failed:", e)
    except ValidationError as ve:
        print("❌ Validation failed:", ve)

except Exception as e:
    print("OpenAI API error:", e)

Raw output:
 {
  "employees": [
    {
      "name": "John Doe",
      "age": 32,
      "city": "New York"
    },
    {
      "name": "Jane Smith",
      "age": 28,
      "city": "Chicago"
    },
    {
      "name": "Robert Johnson",
      "age": 45,
      "city": "Los Angeles"
    },
    {
      "name": "Elizabeth Davis",
      "age": 38,
      "city": "San Francisco"
    },
    {
      "name": "James Brown",
      "age": 50,
      "city": "Houston"
    }
  ]
}
❌ Validation failed: 1 validation error for EmployeeDict
employees
  Input should be a valid dictionary or instance of PersonInfo [type=model_type, input_value=[{'name': 'John Doe', 'ag... 50, 'city': 'Houston'}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
